In [1]:
# Let's make sure these directories are clean before we start
import shutil
try:
    shutil.rmtree("data/Hurricane-split/train")
    shutil.rmtree("data/Hurricane-split/test")
except:
    pass

In [2]:
import os
# ensure directories exist
from pathlib import Path

Path("data/Hurricane-split/train/damage").mkdir(parents=True, exist_ok=True)
Path("data/Hurricane-split/train/no_damage").mkdir(parents=True, exist_ok=True)

Path("data/Hurricane-split/test/damage").mkdir(parents=True, exist_ok=True)
Path("data/Hurricane-split/test/no_damage").mkdir(parents=True, exist_ok=True)

In [3]:
all_damage_file_paths = os.listdir('data/data_all_modified/damage')
all_no_damage_file_paths = os.listdir('data/data_all_modified/no_damage')

In [4]:
import random

train_damage_paths = random.sample(all_damage_file_paths, int(len(all_damage_file_paths)*0.8))
print("train damage image count: ", len(train_damage_paths))
test_damage_paths = [ p for p in all_damage_file_paths if p not in train_damage_paths]
print("test damage image count: ", len(test_damage_paths))
# ensure no overlap:
overlap = [p for p in train_damage_paths if p in test_damage_paths]
print("len of overlap: ", len(overlap))

train_no_damage_paths = random.sample(all_no_damage_file_paths, int(len(all_no_damage_file_paths)*0.8))
print("train no damage image count: ", len(train_no_damage_paths))
test_no_damage_paths = [ p for p in all_no_damage_file_paths if p not in train_no_damage_paths]
print("test no damage image count: ", len(test_no_damage_paths))
# ensure no overlap:
overlap = [p for p in train_no_damage_paths if p in test_no_damage_paths]
print("len of overlap: ", len(overlap))

train damage image count:  11336
test damage image count:  2834
len of overlap:  0
train no damage image count:  5721
test no damage image count:  1431
len of overlap:  0


In [5]:
import shutil
for p in train_damage_paths:
    shutil.copyfile(os.path.join('data/data_all_modified/damage', p), os.path.join('data/Hurricane-split/train/damage', p) )

for p in test_damage_paths:
    shutil.copyfile(os.path.join('data/data_all_modified/damage', p), os.path.join('data/Hurricane-split/test/damage', p) )

for p in train_no_damage_paths:
    shutil.copyfile(os.path.join('data/data_all_modified/no_damage', p), os.path.join('data/Hurricane-split/train/no_damage', p) )

for p in test_no_damage_paths:
    shutil.copyfile(os.path.join('data/data_all_modified/no_damage', p), os.path.join('data/Hurricane-split/test/no_damage', p) )


In [6]:
print("Files in train/damage: ", len(os.listdir("data/Hurricane-split/train/damage")))
print("Files in train/no_damage: ", len(os.listdir("data/Hurricane-split/train/no_damage")))

print("Files in test/damage: ", len(os.listdir("data/Hurricane-split/test/damage")))
print("Files in test/no_damage: ", len(os.listdir("data/Hurricane-split/test/no_damage")))

Files in train/damage:  11336
Files in train/no_damage:  5721
Files in test/damage:  2834
Files in test/no_damage:  1431


In [7]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
train_data_dir = 'data/Hurricane-split/train/'

batch_size = 32

# target image size
img_height = 150
img_width = 150

train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
train_data_dir,
validation_split=0.2,
subset="both",
seed=123,
image_size=(img_height, img_width),
batch_size=batch_size
)
rescale = Rescaling(scale=1.0/255)
train_rescale_ds = train_ds.map(lambda image,label:(rescale(image),label))
val_rescale_ds = val_ds.map(lambda image,label:(rescale(image),label))

2024-04-11 20:08:23.864799: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-11 20:08:23.903375: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-11 20:08:23.903409: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-11 20:08:23.904555: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-11 20:08:23.911155: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-11 20:08:23.912248: I tensorflow/core/platform/cpu_feature_guard.cc:1

Found 17057 files belonging to 2 classes.
Using 13646 files for training.
Using 3411 files for validation.


In [8]:
test_data_dir = 'data/Hurricane-split/test/'

batch_size = 2

img_height = 150
img_width = 150

test_ds = tf.keras.utils.image_dataset_from_directory(
test_data_dir,
seed=123,
image_size=(img_height, img_width),
)

# rescale the data
rescale = Rescaling(scale=1.0/255)
test_rescale_ds = test_ds.map(lambda image,label:(rescale(image),label))

Found 4265 files belonging to 2 classes.


# ANN with sigmoid activation function

In [23]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten

# create model
model1 = Sequential()

model1.add(Flatten(input_shape=(img_height,img_width,3)))
# input layer
model1.add(Dense(120,activation='relu',input_shape=(img_height*img_width,)))

# Hidden layer
model1.add(Dense(128, activation='relu'))

# output layer (2 labels)
model1.add(Dense(1, activation='sigmoid'))

model1.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_6 (Flatten)         (None, 67500)             0         
                                                                 
 dense_18 (Dense)            (None, 120)               8100120   
                                                                 
 dense_19 (Dense)            (None, 128)               15488     
                                                                 
 dense_20 (Dense)            (None, 1)                 129       
                                                                 
Total params: 8115737 (30.96 MB)
Trainable params: 8115737 (30.96 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model1.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=20,
            validation_data=val_rescale_ds
)

results_test = model1.evaluate(test_rescale_ds, batch_size = 128)
print(results_test)

Epoch 1/20
427/427 [==============================] - 17s 39ms/step - loss: 0.9028 - accuracy: 0.6427 - val_loss: 0.6987 - val_accuracy: 0.6711
Epoch 2/20
427/427 [==============================] - 16s 38ms/step - loss: 0.6072 - accuracy: 0.6959 - val_loss: 0.5600 - val_accuracy: 0.7215
Epoch 3/20
427/427 [==============================] - 16s 38ms/step - loss: 0.6053 - accuracy: 0.6663 - val_loss: 0.5964 - val_accuracy: 0.6681
Epoch 4/20
427/427 [==============================] - 16s 38ms/step - loss: 0.5978 - accuracy: 0.6637 - val_loss: 0.5966 - val_accuracy: 0.6681
Epoch 5/20
427/427 [==============================] - 16s 38ms/step - loss: 0.5979 - accuracy: 0.6611 - val_loss: 0.5684 - val_accuracy: 0.6681
Epoch 6/20
427/427 [==============================] - 16s 38ms/step - loss: 0.5803 - accuracy: 0.6899 - val_loss: 0.6787 - val_accuracy: 0.4186
Epoch 7/20
427/427 [==============================] - 16s 37ms/step - loss: 0.5795 - accuracy: 0.7097 - val_loss: 0.6174 - val_accuracy:

# ANN with tanh activation function

In [21]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten

# create model
model2= Sequential()

model2.add(Flatten(input_shape=(img_height,img_width,3)))
# input layer
model2.add(Dense(120,activation='relu',input_shape=(img_height*img_width,)))

# Hidden layer
model2.add(Dense(128, activation='relu'))

# output layer (2 labels)
model2.add(Dense(2, activation='tanh'))

model2.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 67500)             0         
                                                                 
 dense_15 (Dense)            (None, 120)               8100120   
                                                                 
 dense_16 (Dense)            (None, 128)               15488     
                                                                 
 dense_17 (Dense)            (None, 2)                 258       
                                                                 
Total params: 8115866 (30.96 MB)
Trainable params: 8115866 (30.96 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model2.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=15,
            validation_data=val_rescale_ds
)

results_test = model2.evaluate(test_rescale_ds, batch_size = 128)
print(results_test)

Epoch 1/15
375/427 [=========================>....] - ETA: 1s - loss: 5.1657 - accuracy: 0.6648

KeyboardInterrupt: 

# ANN with softmax activation function

In [9]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten

# create model
model = Sequential()

model.add(Flatten(input_shape=(img_height,img_width,3)))
# input layer
model.add(Dense(120,activation='relu',input_shape=(img_height*img_width,)))

# Hidden layer
model.add(Dense(128, activation='relu'))

# output layer (2 labels)
model.add(Dense(2, activation='softmax'))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 67500)             0         
                                                                 
 dense (Dense)               (None, 120)               8100120   
                                                                 
 dense_1 (Dense)             (None, 128)               15488     
                                                                 
 dense_2 (Dense)             (None, 2)                 258       
                                                                 
Total params: 8115866 (30.96 MB)
Trainable params: 8115866 (30.96 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=30,
            validation_data=val_rescale_ds
)

Epoch 1/25
410/427 [===========================>..] - ETA: 0s - loss: 0.9051 - accuracy: 0.6336

KeyboardInterrupt: 

In [36]:
test_loss, test_accuracy = model.evaluate(test_rescale_ds, verbose=0)

In [37]:
test_accuracy

0.7193434834480286

# Lenet 5 with softmax activation function

In [31]:
from keras import layers
from keras import models
import pandas as pd
from tensorflow.keras.layers import Flatten, Dense
from keras import optimizers
from tensorflow.keras import Sequential

model_lenet5 = models.Sequential()

# Layer 1: Convolutional layer with 6 filters of size 3x3, followed by average pooling
model_lenet5.add(layers.Conv2D(6, kernel_size=(3, 3), activation='relu', input_shape=(img_height,img_width,3)))
model_lenet5.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Layer 2: Convolutional layer with 16 filters of size 3x3, followed by average pooling
model_lenet5.add(layers.Conv2D(16, kernel_size=(3, 3), activation='relu'))
model_lenet5.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Flatten the feature maps to feed into fully connected layers
model_lenet5.add(Flatten())

# Layer 3: Fully connected layer with 120 neurons
model_lenet5.add(layers.Dense(120, activation='relu'))

# Layer 4: Fully connected layer with 84 neurons
model_lenet5.add(layers.Dense(84, activation='relu'))

# Output layer (2 labels)
model_lenet5.add(layers.Dense(2, activation='softmax'))

# Compile model
model_lenet5.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Generating the summary of the model
model_lenet5.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 148, 148, 6)       168       
                                                                 
 average_pooling2d_4 (Avera  (None, 74, 74, 6)         0         
 gePooling2D)                                                    
                                                                 
 conv2d_5 (Conv2D)           (None, 72, 72, 16)        880       
                                                                 
 average_pooling2d_5 (Avera  (None, 36, 36, 16)        0         
 gePooling2D)                                                    
                                                                 
 flatten_9 (Flatten)         (None, 20736)             0         
                                                                 
 dense_27 (Dense)            (None, 120)              

In [16]:
history = model_lenet5.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=30,
            validation_data=val_rescale_ds
)

Epoch 1/30
427/427 [==============================] - 40s 93ms/step - loss: 0.6030 - accuracy: 0.6932 - val_loss: 0.5118 - val_accuracy: 0.8091
Epoch 2/30
427/427 [==============================] - 40s 93ms/step - loss: 0.4933 - accuracy: 0.7824 - val_loss: 0.4246 - val_accuracy: 0.8563
Epoch 3/30
427/427 [==============================] - 37s 88ms/step - loss: 0.4323 - accuracy: 0.8246 - val_loss: 0.4845 - val_accuracy: 0.7954
Epoch 4/30
427/427 [==============================] - 37s 87ms/step - loss: 0.4066 - accuracy: 0.8391 - val_loss: 0.4301 - val_accuracy: 0.8279
Epoch 5/30
427/427 [==============================] - 37s 86ms/step - loss: 0.3849 - accuracy: 0.8509 - val_loss: 0.3597 - val_accuracy: 0.8634
Epoch 6/30
427/427 [==============================] - 36s 84ms/step - loss: 0.3699 - accuracy: 0.8592 - val_loss: 0.3693 - val_accuracy: 0.8531
Epoch 7/30
427/427 [==============================] - 36s 84ms/step - loss: 0.3452 - accuracy: 0.8688 - val_loss: 0.3371 - val_accuracy:

In [30]:
results_test = model_lenet5.evaluate(test_rescale_ds, batch_size = 128)
print(results_test)

NameError: name 'model_lenet5' is not defined

In [21]:
results_train = model_lenet5.evaluate(train_rescale_ds, batch_size = 128)
print(results_train)

427/427 [==============================] - 10s 24ms/step - loss: 0.0661 - accuracy: 0.9802
[0.06614977866411209, 0.98021399974823]


# Lenet-5 With Sigmoid Actiation Function

In [27]:
from keras import layers
from keras import models
import pandas as pd
from tensorflow.keras.layers import Flatten, Dense
from keras import optimizers
from tensorflow.keras import Sequential

model2_lenet5 = models.Sequential()

# Layer 1: Convolutional layer with 6 filters of size 3x3, followed by average pooling
model2_lenet5.add(layers.Conv2D(6, kernel_size=(3, 3), activation='relu', input_shape=(img_height,img_width,3)))
model2_lenet5.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Layer 2: Convolutional layer with 16 filters of size 3x3, followed by average pooling
model2_lenet5.add(layers.Conv2D(16, kernel_size=(3, 3), activation='relu'))
model2_lenet5.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Flatten the feature maps to feed into fully connected layers
model2_lenet5.add(Flatten())

# Layer 3: Fully connected layer with 120 neurons
model2_lenet5.add(layers.Dense(120, activation='relu'))

# Layer 4: Fully connected layer with 84 neurons
model2_lenet5.add(layers.Dense(84, activation='relu'))

# Output layer (2 labels)
model2_lenet5.add(layers.Dense(1, activation='sigmoid'))

# Compile model
model2_lenet5.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Generating the summary of the model
model2_lenet5.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 148, 148, 6)       168       
                                                                 
 average_pooling2d_2 (Avera  (None, 74, 74, 6)         0         
 gePooling2D)                                                    
                                                                 
 conv2d_3 (Conv2D)           (None, 72, 72, 16)        880       
                                                                 
 average_pooling2d_3 (Avera  (None, 36, 36, 16)        0         
 gePooling2D)                                                    
                                                                 
 flatten_8 (Flatten)         (None, 20736)             0         
                                                                 
 dense_24 (Dense)            (None, 120)              

In [28]:
history = model2_lenet5.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=30,
            validation_data=val_rescale_ds
)

Epoch 1/20
427/427 [==============================] - 35s 80ms/step - loss: 0.6186 - accuracy: 0.6831 - val_loss: 0.5423 - val_accuracy: 0.7875
Epoch 2/20
427/427 [==============================] - 35s 82ms/step - loss: 0.4877 - accuracy: 0.7938 - val_loss: 0.5365 - val_accuracy: 0.7147
Epoch 3/20
427/427 [==============================] - 35s 82ms/step - loss: 0.4008 - accuracy: 0.8424 - val_loss: 0.4754 - val_accuracy: 0.7675
Epoch 4/20
427/427 [==============================] - 35s 83ms/step - loss: 0.3668 - accuracy: 0.8593 - val_loss: 0.3572 - val_accuracy: 0.8736
Epoch 5/20
427/427 [==============================] - 35s 82ms/step - loss: 0.3547 - accuracy: 0.8645 - val_loss: 0.4166 - val_accuracy: 0.8138
Epoch 6/20
427/427 [==============================] - 35s 82ms/step - loss: 0.3423 - accuracy: 0.8692 - val_loss: 0.3444 - val_accuracy: 0.8648
Epoch 7/20
427/427 [==============================] - 35s 82ms/step - loss: 0.3341 - accuracy: 0.8749 - val_loss: 0.3376 - val_accuracy:

In [29]:
results_test = model2_lenet5.evaluate(test_rescale_ds, batch_size = 128)
print(results_test)

134/134 [==============================] - 3s 24ms/step - loss: 0.2312 - accuracy: 0.9109
[0.23121997714042664, 0.9109026789665222]


# Lenet-5 With Tanh Activation Function

In [32]:
from keras import layers
from keras import models
import pandas as pd
from tensorflow.keras.layers import Flatten, Dense
from keras import optimizers
from tensorflow.keras import Sequential

model3_lenet5 = models.Sequential()

# Layer 1: Convolutional layer with 6 filters of size 3x3, followed by average pooling
model3_lenet5.add(layers.Conv2D(6, kernel_size=(3, 3), activation='relu', input_shape=(img_height,img_width,3)))
model3_lenet5.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Layer 2: Convolutional layer with 16 filters of size 3x3, followed by average pooling
model3_lenet5.add(layers.Conv2D(16, kernel_size=(3, 3), activation='relu'))
model3_lenet5.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Flatten the feature maps to feed into fully connected layers
model3_lenet5.add(Flatten())

# Layer 3: Fully connected layer with 120 neurons
model3_lenet5.add(layers.Dense(120, activation='relu'))

# Layer 4: Fully connected layer with 84 neurons
model3_lenet5.add(layers.Dense(84, activation='relu'))

# Output layer (2 labels)
model3_lenet5.add(layers.Dense(2, activation='tanh'))

# Compile model
model3_lenet5.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Generating the summary of the model
model3_lenet5.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 148, 148, 6)       168       
                                                                 
 average_pooling2d_6 (Avera  (None, 74, 74, 6)         0         
 gePooling2D)                                                    
                                                                 
 conv2d_7 (Conv2D)           (None, 72, 72, 16)        880       
                                                                 
 average_pooling2d_7 (Avera  (None, 36, 36, 16)        0         
 gePooling2D)                                                    
                                                                 
 flatten_10 (Flatten)        (None, 20736)             0         
                                                                 
 dense_30 (Dense)            (None, 120)             

In [ ]:
history = model3_lenet5.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=30,
            validation_data=val_rescale_ds
)

Epoch 1/30
427/427 [==============================] - 36s 81ms/step - loss: 0.5818 - accuracy: 0.5273 - val_loss: 0.4948 - val_accuracy: 0.5561
Epoch 2/30
427/427 [==============================] - 35s 82ms/step - loss: 0.4833 - accuracy: 0.4966 - val_loss: 0.4027 - val_accuracy: 0.3412
Epoch 3/30
427/427 [==============================] - 35s 83ms/step - loss: 0.4211 - accuracy: 0.4464 - val_loss: 0.5405 - val_accuracy: 0.2870
Epoch 4/30
427/427 [==============================] - 35s 82ms/step - loss: 0.3604 - accuracy: 0.4391 - val_loss: 0.4182 - val_accuracy: 0.2568
Epoch 5/30
116/427 [=======>......................] - ETA: 23s - loss: 0.3555 - accuracy: 0.4316

# Alternate Lenet-5 

In [22]:
from keras import layers
from keras import models
import pandas as pd
from tensorflow.keras.layers import Flatten, Dense
from keras import optimizers
from tensorflow.keras import Sequential

model_alt_lenet5 = models.Sequential()

# 2D convolutional layer with 32 filters of size 3x3
model_alt_lenet5.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(img_height,img_width,3)))
# 2D Max pooling layer with 2x2 pooling size
model_alt_lenet5.add(layers.MaxPooling2D(pool_size=(2, 2)))

# 2D convolutional layer with 32 filters of size 3x3 
model_alt_lenet5.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
# 2D Max pooling layer with 2x2 pooling size
model_alt_lenet5.add(layers.MaxPooling2D(pool_size=(2, 2)))

# 2D convolutional layer with 128 filters of size 3x3
model_alt_lenet5.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
# 2D Max pooling layer with 2x2 pooling size
model_alt_lenet5.add(layers.MaxPooling2D(pool_size=(2, 2)))

# 2D convolutional layer with 128 filters of size 3x3
model_alt_lenet5.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
# 2D Max pooling layer with 2x2 pooling size
model_alt_lenet5.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Flatten to feed into fully connected layers
model_alt_lenet5.add(layers.Flatten())

# 50% dropout
model_alt_lenet5.add(layers.Dropout(0.5))

# Fully connected layer with 512 neurons
model_alt_lenet5.add(layers.Dense(512, activation='relu'))

# Fully connected layer with 2 neurons
model_alt_lenet5.add(layers.Dense(2, activation='softmax'))

model_alt_lenet5.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model_alt_lenet5.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 conv2d_3 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 17, 17, 128)      

In [23]:
history = model_alt_lenet5.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=25,
            validation_data=val_rescale_ds
)

Epoch 1/25
427/427 [==============================] - 136s 317ms/step - loss: 0.5431 - accuracy: 0.7371 - val_loss: 0.4000 - val_accuracy: 0.8259
Epoch 2/25
427/427 [==============================] - 138s 322ms/step - loss: 0.3817 - accuracy: 0.8390 - val_loss: 0.3088 - val_accuracy: 0.8789
Epoch 3/25
427/427 [==============================] - 141s 331ms/step - loss: 0.2906 - accuracy: 0.8837 - val_loss: 0.3106 - val_accuracy: 0.8854
Epoch 4/25
427/427 [==============================] - 148s 346ms/step - loss: 0.2119 - accuracy: 0.9170 - val_loss: 0.1791 - val_accuracy: 0.9279
Epoch 5/25
427/427 [==============================] - 142s 332ms/step - loss: 0.1666 - accuracy: 0.9348 - val_loss: 0.3686 - val_accuracy: 0.8440
Epoch 6/25
427/427 [==============================] - 134s 313ms/step - loss: 0.1465 - accuracy: 0.9421 - val_loss: 0.1943 - val_accuracy: 0.9252
Epoch 7/25
427/427 [==============================] - 138s 324ms/step - loss: 0.1304 - accuracy: 0.9470 - val_loss: 0.1450 -

In [24]:
results_test = model_alt_lenet5.evaluate(test_rescale_ds, batch_size = 128)
print(results_test)

134/134 [==============================] - 9s 65ms/step - loss: 0.0497 - accuracy: 0.9831
[0.049680184572935104, 0.9831184148788452]


In [25]:
model_alt_lenet5.save("hurricane.keras")

# Testing the Inference Server

In [28]:
import tensorflow as tf
final_model = tf.keras.models.load_model('models/hurricane.keras')

In [29]:
results_test = final_model.evaluate(test_rescale_ds, batch_size=128)
print(results_test)

134/134 [==============================] - 9s 66ms/step - loss: 0.0377 - accuracy: 0.9880
[0.037746842950582504, 0.9880421757698059]


In [33]:
# from chatGPT
labels_dataset = test_rescale_ds.map(lambda features, label:label)
labels = list(labels_dataset.as_numpy_iterator())
labels[0]

array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0], dtype=int32)

In [31]:
# from chatGPT
my_iterator = iter(test_rescale_ds)
first_entry = my_iterator.get_next()[0].numpy()
l = first_entry.tolist()

In [32]:
import requests
import json

rsp = requests.post("http://172.17.0.1:5000/models/hurricane/v1", json={"image": l})

rsp.json()

{'result': [[0.9999983906745911, 1.5237205843732227e-06],
  [0.0007256282260641456, 0.9992744326591492],
  [3.139949455999158e-08, 0.9999999403953552],
  [3.8863421991663927e-07, 0.9999995827674866],
  [0.9999603629112244, 3.958683737437241e-05],
  [1.7940717498277081e-06, 0.999998152256012],
  [0.9999273419380188, 7.257550896611065e-05],
  [0.9981266856193542, 0.0018732883036136627],
  [0.004130591172724962, 0.9958693385124207],
  [0.9999237656593323, 7.618772360729054e-05],
  [0.08077516406774521, 0.9192247986793518],
  [0.00018121047469321638, 0.9998188018798828],
  [0.010800685733556747, 0.9891993403434753],
  [0.9999762177467346, 2.367173510720022e-05],
  [0.008514531888067722, 0.991485595703125],
  [0.9999434351921082, 5.651496030623093e-05],
  [0.017641067504882812, 0.982358992099762],
  [8.308465027084821e-08, 0.9999998211860657],
  [0.9989575743675232, 0.001042312360368669],
  [0.9998290538787842, 0.00017098280659411103],
  [3.728641604539007e-05, 0.9999626278877258],
  [0.999

In [34]:
final_model.predict(l).tolist()

1/1 [==============================] - 0s 199ms/step


[[0.9999983906745911, 1.5237205843732227e-06],
 [0.0007256282260641456, 0.9992744326591492],
 [3.139949455999158e-08, 0.9999999403953552],
 [3.8863421991663927e-07, 0.9999995827674866],
 [0.9999603629112244, 3.958683737437241e-05],
 [1.7940717498277081e-06, 0.999998152256012],
 [0.9999273419380188, 7.257550896611065e-05],
 [0.9981266856193542, 0.0018732883036136627],
 [0.004130591172724962, 0.9958693385124207],
 [0.9999237656593323, 7.618772360729054e-05],
 [0.08077516406774521, 0.9192247986793518],
 [0.00018121047469321638, 0.9998188018798828],
 [0.010800685733556747, 0.9891993403434753],
 [0.9999762177467346, 2.367173510720022e-05],
 [0.008514531888067722, 0.991485595703125],
 [0.9999434351921082, 5.651496030623093e-05],
 [0.017641067504882812, 0.982358992099762],
 [8.308465027084821e-08, 0.9999998211860657],
 [0.9989575743675232, 0.001042312360368669],
 [0.9998290538787842, 0.00017098280659411103],
 [3.728641604539007e-05, 0.9999626278877258],
 [0.9998636245727539, 0.000136529648443